# Read Offline from CSV

## Overview
Data manually downloaded from

 - **WSJ.com**
  
 - **Investing.com**

  
### Offline Data

| No | Symbol | Index Name | Source | URL Address |
| :--- | :----- | :----- | :---- | :----- |
| 1 | COMEX_Gold | Gold: Future closing price | Investing | https://www.investing.com/commodities/gold-historical-data |
| 2 | DXY | US Dollar Index  | Investing | https://www.investing.com/indices/usdollar-historical-data |
| 3 | LME_Copper | Copper: Future closing price | Investing | https://www.investing.com/commodities/copper-historical-data |
| 4 | COMP | NASDAQ index | WSJ | https://www.wsj.com/market-data/quotes/index/COMP/historical-prices |
| 5 | DJIA | Dow Jones Industrial Index | WSJ | https://www.wsj.com/market-data/quotes/index/DJIA/historical-prices |
| 6 | SPX | S&P 500 index | WSJ | https://www.wsj.com/market-data/quotes/index/SPX/historical-prices |


---
### Data fields

WSJ: `['Date', 'Open', 'High', 'Low', 'Close']`

Investing:`['Date', 'Price', 'Open','High','Low','Vol','Change']`

## Import Libraries

In [1]:
# import necessary libraries
import pandas as pd
import os
import glob

import pickle

## Custom functions

In [2]:
def get_file_name(file_name):
    '''
    Get full file address and return the name
    '''
    name_ext = file_name.split("\\")[-1]
    name = name_ext.split(".")[0]
    
    return name

In [3]:
def monthly_average(df):
    '''
    Get a dataframe with daily records 
    and return a new df with monthly average
    '''
    df['month'] = pd.DatetimeIndex(df.index).month
    df['year'] = pd.DatetimeIndex(df.index).year
    
    dfg = df.groupby(['year','month'], as_index=False).mean()
    dfg['Date'] = pd.to_datetime(dict(year = dfg.year, month = dfg.month, day = 1))
    
    dfg = dfg.drop(columns = ['month','year'])
    
    dfg.set_index('Date', drop=True, inplace=True)
    
    return dfg

## Set Directories

In [4]:
# Change directory one step back and save as the root directory
ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Change direcotry to the 'data\raw' folder
path = ROOT_DIR + '\\data\\raw\\'
print(path)

D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01\data\raw\


In [5]:
# use glob to get all the csv files in the folder
csv_files = glob.glob(os.path.join(path, "*.csv"))
#dispaly csv file list
display(csv_files)

['D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\COMP.csv',
 'D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\Copper_Futures.csv',
 'D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\DJIA.csv',
 'D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\DXY.csv',
 'D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\Gold_Futures.csv',
 'D:\\gitProjects\\springboard_capstone_1\\Springboard_Capstone_01\\data\\raw\\SPX.csv']

## Load all CSV files

In [6]:
# Initialize list; this is the final list that will store all the data from the json pull. 
read_data = []

# loop over the list of csv files
for f in csv_files:
        
    # read the csv file    
    df = pd.read_csv(f, sep = ',', thousands=',')
    
    if ' Close' in df.columns:
        #rename column name
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close']
        value = 'Close'
        format_date = '%m/%d/%y'
        
        
    if 'Price' in df.columns:
        df.columns = ['Date', 'Price', 'Open','High','Low','Vol','Change']
        value = 'Price'
        format_date = '%m/%d/%Y'
    
    #select columns
    df = df[['Date', value]]

    #change data type to date and numeric
    df['value'] = pd.to_numeric(df[value])
    df['Date'] = pd.to_datetime(df['Date'], format=format_date, errors="raise")
    
    df = df.drop(columns = [value])
    df = df.rename(columns = {'value': get_file_name(f)}, errors="raise")
    
    #set Date as index
    df.set_index('Date', drop=True, inplace=True)
    
    print('Sucess:', get_file_name(f))
    #display(df.head())
    
    read_data.append(df)
    # print the success report
    


Sucess: COMP
Sucess: Copper_Futures
Sucess: DJIA
Sucess: DXY
Sucess: Gold_Futures
Sucess: SPX


## Data

### Print the features and sizes

In [7]:
for i in range(len(read_data)):
    name = read_data[i].columns[0]
    size = len(read_data[i])
    phrase = str(i) + ': '+ name + ' (1*' + str(size) + ')'
    print(phrase)

0: COMP (1*5738)
1: Copper_Futures (1*274)
2: DJIA (1*5737)
3: DXY (1*274)
4: Gold_Futures (1*274)
5: SPX (1*5737)


### Merge_order all data frames

In [8]:
df = read_data[0]
dfg = monthly_average(df)
dfg.index = pd.to_datetime(dfg.index, format = '%Y%m%d',errors='coerce')

for df_temp in read_data[1:]:
    dfg_temp = monthly_average(df_temp)
    dfg_temp.index = pd.to_datetime(dfg_temp.index, format = '%Y%m%d',errors='coerce')
    dfg = pd.merge_ordered(dfg, dfg_temp, on = 'Date')

## Check the final dataframe

In [9]:
display(dfg)

,Date,COMP,Copper_Futures,DJIA,DXY,Gold_Futures,SPX
0,2000-01-01,4013.494000,0.8545,11281.263000,105.13,286.20,1425.585500
1,2000-02-01,4410.871000,0.7980,10541.929500,105.92,295.40,1388.874000
2,2000-03-01,4802.988261,0.8085,10483.390435,105.44,281.40,1442.213043
3,2000-04-01,3863.637368,0.7960,10944.310526,110.14,276.00,1461.355263
4,2000-05-01,3528.416818,0.8160,10580.267727,108.74,274.80,1418.479545
...,...,...,...,...,...,...,...
269,2022-06-01,11418.919048,3.7100,31446.712857,104.68,1812.00,3898.946667
270,2022-07-01,11622.631500,3.5750,31535.323500,106.00,1771.50,3911.729500
271,2022-08-01,12570.260435,3.5190,33009.564783,108.85,1721.40,4158.563043
272,2022-09-01,11413.214091,3.4125,30649.559524,112.12,1672.00,3850.520476


## Store the data

In [10]:
# Change direcotry to the 'data\raw' folder
save_dir = ROOT_DIR + '\\data\\interim\\'

# Set a file name
file_save = save_dir + 'offline_data_raw.csv'
dfg.to_csv(file_save)

print('Save:\n', file_save)

Save:
 D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01\data\interim\offline_data_raw.csv
